In [1]:
import sys
import pandas as pd
import numpy as np
import os
home = os.getcwd()[:os.getcwd().find('EnsembleCpG')]+'EnsembleCpG/'
os.environ["PYTHONPATH"] = home+"code/"
sys.path[1] = os.environ["PYTHONPATH"]
from common import commons 
home = commons.home
from log import Logger
from heapq import nsmallest

INFO    ; 2019-01-04 09:38:21,977; root           ; commons:<module>;39: Existing logger at commons


In [2]:
def cal_beta(beta_file,pos_file):
    betas = pd.read_csv(beta_file,sep='\s+',index_col=['TargetID'])
    mean_betas = pd.DataFrame(betas.mean(axis=1),columns=['beta'])
    mean_betas.index = betas.index
    pos = pd.read_csv(pos_file,sep='\s+',usecols=[0,2,3],index_col=0, header=None,skiprows=1,names=['id','chr','coordinate'])
    beta_pos = mean_betas.join(pos)
    return beta_pos

In [3]:
log_dir = home+'logs/'
logger = Logger.Logger(log_dir).get_logger()
beta_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_imputed.tsv'
pos_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_metaData.tsv'
all_sites_betas = cal_beta(beta_file,pos_file)

In [4]:
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
pos_pvalues ={'amyloid':0.00005,'cerad':0.00001,'ceradaf':0.00005,'tangles':0.0000005,'cogdec':0.00003,'gpath':0.00001,'braak':0.00005}
pos_pvalue = pos_pvalues[type_name]  ### 0.001 for amyloid, 0.0001 for cerad, 0.00001 for tangles,0.002 for cogdec, 0.0002 for gpath,0.0002 for braak
neg_pvalue = 0.4
sample_ratio_neg_to_pos = 10

In [5]:
all_sites_file = home+'data/AD_CpG/Rosmap_'+type_name+'_ewas_'+with_cell_type+'celltype.csv'
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)
all_sites = all_sites[['id','chr','coordinate','beta_sign','pvalue','beta']]
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)
positive_sites = all_sites.query('pvalue<=@pos_pvalue')
positive_sites['label'] = np.where(positive_sites['beta_sign']>0,1,-1)
negative_sites = all_sites.query('pvalue>@neg_pvalue')
negative_sites['label'] = 0
negatives_sort_by_beta = negative_sites.sort_values(['beta'])

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
positive_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label
119979,cg05731218,2,216769199.0,34.784645,5.741779e-09,0.859388,1
39932,cg05066959,8,41519308.0,33.098300,1.312081e-08,0.865159,1
97325,cg06811467,1,154299179.0,30.751567,4.165130e-08,0.085489,1
41616,cg17693222,8,42033472.0,28.726615,1.134165e-07,0.867586,1
60280,cg07883124,13,113634042.0,28.609516,1.201982e-07,0.705831,1
40821,cg11823178,8,41519399.0,27.901517,1.708237e-07,0.919399,1
38784,cg25594100,7,4786943.0,26.998223,2.677289e-07,0.780079,1
157640,cg25446460,4,176486240.0,26.851843,2.879790e-07,0.898800,1
103395,cg13390284,1,65531864.0,26.579965,3.297668e-07,0.128000,1
142475,cg16098726,3,128778738.0,26.134805,4.117633e-07,0.836880,1


In [21]:
select_negs_list = []
hyper_sites = negatives_sort_by_beta.query('beta_sign>=0')
hypo_sites = negatives_sort_by_beta.query('beta_sign<0')
for beta,beta_sign in positive_sites[['beta','beta_sign']].values:
    tmp_sites = hyper_sites if beta_sign >=0 else hypo_sites
    negs = tmp_sites.loc[nsmallest(10, tmp_sites.index.values, key=lambda i: abs(tmp_sites.loc[i,'beta']-beta)),:]
    select_negs_list.extend(negs.values)
select_negs = pd.DataFrame(select_negs_list,columns=['id','chr','coordinate','beta_sign','pvalue','beta','label']).drop_duplicates(['chr','coordinate'])

In [20]:
select_negs

,id,chr,coordinate,beta_sign,pvalue,beta,label
0,cg15081033,3,140805567.0,0.466975,0.494609,0.859381,0
1,cg14599015,1,205210699.0,0.000795,0.977510,0.859382,0
2,cg09267507,17,26671495.0,0.569556,0.450690,0.859383,0
3,cg09452147,10,17713581.0,0.239800,0.624505,0.859384,0
4,cg20984614,8,144774308.0,0.314151,0.575324,0.859386,0
5,cg18683707,5,148754068.0,0.000001,0.999088,0.859394,0
6,cg04504715,8,101718986.0,0.334652,0.563119,0.859395,0
7,cg22110569,1,161009463.0,0.043218,0.835377,0.859396,0
8,cg12908665,5,114499431.0,0.042309,0.837092,0.859401,0
9,cg11832000,10,61153075.0,0.644372,0.422406,0.859403,0


In [8]:
win_path = home+'data/commons/wins.txt'
pos_sites_with_winid, neg_sites_with_winid = commons.merge_with_feature_windows(win_path,positive_sites,select_negs)
all_sites_with_winid = pos_sites_with_winid.append(neg_sites_with_winid,ignore_index=True)
all_sites_with_winid.drop_duplicates(['id'],inplace=True)
all_sites_with_winid.sort_values(['chr','coordinate'],inplace=True) 

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  f

In [9]:
#export all features to file
if not os.path.exists(home+'data/AD_CpG/'+type_name+with_cell_type):
    os.mkdir(home+'data/AD_CpG/'+type_name+with_cell_type)
with pd.HDFStore(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_sites_winid','w') as h5s:
    h5s['all_sites_winid'] = all_sites_with_winid       
all_sites_with_winid.to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_sites_winid.csv',index=False)  
all_sites_with_winid['winid'].to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/selected_pos_winid.csv',index=False)

In [10]:
##export winid with all 450k sites
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
all_sites_file = home+'data/AD_CpG/Rosmap_'+type_name+'_ewas_'+with_cell_type+'celltype.csv'
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)
temp = pd.DataFrame()
temp['id'],temp['chr'],temp['coordinate'],temp['beta_sign'],temp['pvalue'],temp['beta'] = all_sites['id'],all_sites['chr'],all_sites['coordinate'],all_sites['beta_sign'],all_sites['pvalue'],all_sites['beta']
all_sites = temp
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)

In [11]:
win_path = home+'data/commons/wins.txt'
all_450k_sites_with_winid, __ = commons.merge_with_feature_windows(win_path,all_sites)
all_450k_sites_with_winid.drop(['beta_sign'],axis=1,inplace=True)

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  f

In [12]:
with pd.HDFStore(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_450k_sites_winid','w') as h5s:
    h5s['all_450k_sites_winid'] = all_450k_sites_with_winid      
all_450k_sites_with_winid.to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_450k_sites_winid.csv',index=False)  
if not os.path.exists(home+'data/AD_CpG/selected_450k_pos_winid.csv'):
    all_450k_sites_with_winid['winid'].to_csv(home+'data/AD_CpG/selected_450k_pos_winid.csv',index=False)
if not os.path.exists(home+'data/AD_CpG/all_450k_sites_winid.csv'):
    all_450k_sites_with_winid.to_csv(home+'data/AD_CpG/all_450k_sites_winid.csv',index=False)
    with pd.HDFStore(home+'data/AD_CpG/all_450k_sites_winid','w') as h5s:
        h5s['all_450k_sites_winid'] = all_450k_sites_with_winid 

In [13]:
all_450k_sites_with_winid

,id,chr,coordinate,pvalue,beta,start,winid,end
0,cg13869341,1,15865.0,0.728959,0.877442,15801,80,16000
1,cg15560884,1,710097.0,0.207597,0.652625,710001,3551,710200
2,cg01014490,1,714177.0,0.945040,0.008987,714001,3571,714200
3,cg17505339,1,720865.0,0.059955,0.859844,720801,3605,721000
4,cg16736630,1,779995.0,0.156873,0.758187,779801,3900,780000
5,cg03128332,1,805338.0,0.858352,0.142779,805201,4027,805400
6,cg18147296,1,812539.0,0.565510,0.677963,812401,4063,812600
7,cg05597748,1,812679.0,0.541403,0.758387,812601,4064,812800
8,cg10988531,1,813317.0,0.277183,0.789541,813201,4067,813400
9,cg13938959,1,834183.0,0.045435,0.864894,834001,4171,834200
